In [1]:
# import Pkg; Pkg.add("JuMP");  Pkg.add("GLPK");  Pkg.add("DelimitedFiles"); 

In [2]:
using JuMP          # Modeling language for optimization
using GLPK          # Open-source solver for linear programming

In [3]:
# Dados do problema
n = 5  # number of tasks
m = 2  # number of workers

2

In [4]:
production_times = [
    3 1 4 1 5
    2 7 1 8 2
]

2×5 Matrix{Int64}:
 3  1  4  1  5
 2  7  1  8  2

In [5]:
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [6]:
# Variables
@variable(model, x[1:n, 1:m], Bin)  # Assignment of tasks to workers
@variable(model, b[1:n, 1:m], Bin)  # Start of a block for a worker
@variable(model, e[1:n, 1:m], Bin)  # End of a block for a worker
@variable(model, T >= 0)            # Maximum workload

T

In [7]:
@objective(model, Min, T)

T

In [8]:
for i in 1:n
    @constraint(model, sum(x[i, j] for j in 1:m) == 1)
end

In [9]:
# Contiguity constraints: Each worker has exactly one contiguous block
for j in 1:m
    # Exactly one start and one end per worker
    @constraint(model, sum(b[i, j] for i in 1:n) == 1)
    @constraint(model, sum(e[i, j] for i in 1:n) == 1)
    
    # Start position definition
    for i in 1:n
        # Handle boundary cases (i=1 and i=num_operations)
        prev_x = (i == 1) ? 0 : x[i-1, j]
        next_x = (i == n) ? 0 : x[i+1, j]
        
        # b[i,j] = 1 if x[i,j] = 1 and x[i-1,j] = 0
        @constraint(model, b[i, j] >= x[i, j] - prev_x)
        @constraint(model, b[i, j] <= x[i, j])
        @constraint(model, b[i, j] <= 1 - prev_x)
        
        # e[i,j] = 1 if x[i,j] = 1 and x[i+1,j] = 0
        @constraint(model, e[i, j] >= x[i, j] - next_x)
        @constraint(model, e[i, j] <= x[i, j])
        @constraint(model, e[i, j] <= 1 - next_x)
    end
end

In [10]:
for j in 1:m
    for i in 1:n
        @constraint(model, b[i, j] + e[i, j] <= 1)
    end
end

In [11]:
# Workload constraints: T >= sum of production times for each worker
for j in 1:m
    @constraint(model, T >= sum(production_times[j, i] * x[i, j] for i in 1:n))
end

In [12]:
optimize!(model)

In [13]:
println("Maximum workload (T): ", objective_value(model))

# Print assignments
for j in 1:m
    assigned_tasks = [i for i in 1:n if value(x[i, j]) > 0.5]
    if !isempty(assigned_tasks)
        println("Worker $j: Tasks ", assigned_tasks, " | Workload: ", sum(production_times[j, i] for i in assigned_tasks))
    end
end

Maximum workload (T): 10.0
Worker 1: Tasks [3, 4, 5] | Workload: 10
Worker 2: Tasks [1, 2] | Workload: 9
